In [1]:
import pymc3
import umap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import time
import os
from scipy.stats import zscore
import threading
import pickle
%matplotlib inline

In [2]:
savedir = "../results/umap_out/4"
if not os.path.exists(savedir):
    os.mkdir(savedir)
README = "Log10 feature values. Z-scored, z-score over 4 outlier removed, and only the 3 features used"
with open("README", "w") as f:
    f.write(README)

## Load in z-scored data

In [3]:
df = pickle.load(open("../data/processed/data_df_log10_z.p","rb"))
df.head()

,Cell Size,Cell Circularity,Cell Aspect Ratio,Cell Tracker Intensity,PI Intensity,AnexinV Intensity
0_2G3,4.514737,-35.280948,4.073811,-0.549886,-1.508243,-1.286249
1_2G3,-0.206776,-23.196179,18.581889,-1.036545,-1.525713,-1.696325
2_2G3,-1.666082,-10.317551,10.892828,-0.955933,-1.518481,-1.653310
3_2G3,-1.082179,-15.445497,13.969401,-0.843089,-1.504416,-1.656892
4_2G3,-3.274187,0.564852,2.419749,-0.857998,-1.514578,-1.643740


### Remove outliers

In [4]:
df = df[(np.abs(df) < 4).all(axis=1)]
df.shape

(25127452, 6)

## Choose
a) n_neighbors - UMAP parameter similar to perplexity in t-sne   
b) min_distance - UMAP parameter reflecting how far apart points can be  
b) n_subsample - number of samples to use from each time-stimulus-genotype combination  
c) n_iter - number of times to run the umap subsample  

In [5]:
n_neighbors_l = [3, 5, 10, 15, 100,200]
min_distance_l = [0.1, 0]
n_subsample = 100
n_iter = 3

In [6]:
meta_df = pd.read_csv("../data/meta.tsv",sep="\t",index_col=0)

In [7]:
df = pd.concat((df,meta_df),axis=1,join='inner')
df.head()

,Cell Size,Cell Circularity,Cell Aspect Ratio,Cell Tracker Intensity,PI Intensity,AnexinV Intensity,Timepoint,Stimuli,Sample,Genotype,Stimuli Names
4_2G3,-3.274187,0.564852,2.419749,-0.857998,-1.514578,-1.643740,3,G,2,WT,Nec1s
5_2G3,-3.274187,0.564852,2.419749,-0.826527,-1.498387,-1.625448,3,G,2,WT,Nec1s
7_2G3,-3.896287,0.564852,-0.858085,-0.761169,-1.525433,-1.595391,3,G,2,WT,Nec1s
17_2G3,-3.896287,0.564852,-0.858085,-0.442464,-1.533792,-1.264574,3,G,2,WT,Nec1s
20_2G3,0.245786,-2.396779,2.406727,-0.277562,-1.489812,-1.210955,3,G,2,WT,Nec1s


## Run umap

## Loop through and run umap
### Use 2 cpus?

In [8]:
attrs = [ "Cell Tracker Intensity", "PI Intensity", "AnexinV Intensity"]

In [9]:
def run_umap(df, f_save, n_neighbors, min_distance, subsample, attrs):
    t1 = time.time()
    reducer = umap.UMAP(n_neighbors=n_neighbors, min_dist=min_distance)
    embedding = reducer.fit_transform(df[attrs])
    embedding.shape
    t2 = time.time()
    print(f"Time in minutes:{(t2-t1)/60}")

    pickle.dump([embedding,df.index],open(f_save.replace(".p","") + ".p",'wb'))
    with open(f_save.replace(".p","") + "_time_minutes.txt","w") as f:
        f.write(str((t2-t1)/60))
    return

In [10]:
df = df.fillna(0)

In [ ]:
for i in range(n_iter):
    print('i',i)
    for neigh in n_neighbors_l:
        print('number of neighbors', neigh)
        for dist in min_distance_l:
            print('minimum distance', dist)
            # File name
            curr_f_save = f"{savedir}/embedding_{i}_{neigh}_{dist}.p" #umap_results
            # Collect samples
            samples = df.groupby(["Stimuli","Genotype","Timepoint"]).apply(lambda x: x.sample(n=n_subsample).reset_index())
            samples = samples.set_index("index")
            run_umap(samples, curr_f_save, neigh, dist, n_subsample, attrs)

i 0
number of neighbors 3
minimum distance 0.1


/home/isshamie/software/anaconda2/envs/umap/lib/python3.7/site-packages/numba/compiler.py:602: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../../../../home/isshamie/software/anaconda2/envs/umap/lib/python3.7/site-packages/umap/rp_tree.py", line 135:
@numba.njit(fastmath=True, nogil=True, parallel=True)
def euclidean_random_projection_split(data, indices, rng_state):
^

  self.func_ir.loc))
/home/isshamie/software/anaconda2/envs/umap/lib/python3.7/site-packages/umap/nndescent.py:92: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for he

Time in minutes:1.9453779617945353
minimum distance 0


/home/isshamie/software/anaconda2/envs/umap/lib/python3.7/site-packages/numba/compiler.py:602: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../../../../home/isshamie/software/anaconda2/envs/umap/lib/python3.7/site-packages/umap/nndescent.py", line 47:
    @numba.njit(parallel=True)
    def nn_descent(
    ^

  self.func_ir.loc))
/home/isshamie/software/anaconda2/envs/umap/lib/python3.7/site-packages/umap/spectral.py:229: UserWarning: Embedding a total of 557 separate connected components using meta-embedding (experimental)
  n_components


Time in minutes:1.9154534498850504
number of neighbors 5
minimum distance 0.1


...

/home/isshamie/software/anaconda2/envs/umap/lib/python3.7/site-packages/numba/compiler.py:602: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../../../../home/isshamie/software/anaconda2/envs/umap/lib/python3.7/site-packages/umap/nndescent.py", line 47:
    @numba.njit(parallel=True)
    def nn_descent(
    ^

  self.func_ir.loc))


Time in minutes:2.8924887140591937
minimum distance 0


/home/isshamie/software/anaconda2/envs/umap/lib/python3.7/site-packages/numba/compiler.py:602: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../../../../home/isshamie/software/anaconda2/envs/umap/lib/python3.7/site-packages/umap/nndescent.py", line 47:
    @numba.njit(parallel=True)
    def nn_descent(
    ^

  self.func_ir.loc))


Time in minutes:1.7045177618662517
number of neighbors 100
minimum distance 0.1


/home/isshamie/software/anaconda2/envs/umap/lib/python3.7/site-packages/numba/compiler.py:602: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../../../../home/isshamie/software/anaconda2/envs/umap/lib/python3.7/site-packages/umap/nndescent.py", line 47:
    @numba.njit(parallel=True)
    def nn_descent(
    ^

  self.func_ir.loc))


In [12]:
n_neighbors_l = [500,1000]
for i in range(n_iter):
    print('i',i)
    for neigh in n_neighbors_l:
        print('number of neighbors', neigh)
        for dist in min_distance_l:
            print('minimum distance', dist)
            # File name
            curr_f_save = f"{savedir}/embedding_{i}_{neigh}_{dist}.p" #umap_results
            if not os.path.exists(curr_f_save):
                print("Running")
                # Collect samples
                samples = df.groupby(["Stimuli","Genotype","Timepoint"]).apply(lambda x: x.sample(n=n_subsample).reset_index())
                samples = samples.set_index("index")
                run_umap(samples, curr_f_save, neigh, dist, n_subsample, attrs)
            else:
                print(f"Already ran {curr_f_save}")

i 0
number of neighbors 500
minimum distance 0.1
Running


/home/isshamie/software/anaconda2/envs/umap/lib/python3.7/site-packages/numba/compiler.py:602: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../../../../home/isshamie/software/anaconda2/envs/umap/lib/python3.7/site-packages/umap/nndescent.py", line 47:
    @numba.njit(parallel=True)
    def nn_descent(
    ^

  self.func_ir.loc))


Time in minutes:9.497182329495748
minimum distance 0
Running


/home/isshamie/software/anaconda2/envs/umap/lib/python3.7/site-packages/numba/compiler.py:602: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../../../../home/isshamie/software/anaconda2/envs/umap/lib/python3.7/site-packages/umap/nndescent.py", line 47:
    @numba.njit(parallel=True)
    def nn_descent(
    ^

  self.func_ir.loc))


Time in minutes:9.114956148465474
number of neighbors 1000
minimum distance 0.1
Running


...

Time in minutes:9.149636312325795
number of neighbors 1000
minimum distance 0.1
Running


/home/isshamie/software/anaconda2/envs/umap/lib/python3.7/site-packages/numba/compiler.py:602: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../../../../home/isshamie/software/anaconda2/envs/umap/lib/python3.7/site-packages/umap/nndescent.py", line 47:
    @numba.njit(parallel=True)
    def nn_descent(
    ^

  self.func_ir.loc))


Time in minutes:18.119917976856232
minimum distance 0
Running


/home/isshamie/software/anaconda2/envs/umap/lib/python3.7/site-packages/numba/compiler.py:602: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../../../../home/isshamie/software/anaconda2/envs/umap/lib/python3.7/site-packages/umap/nndescent.py", line 47:
    @numba.njit(parallel=True)
    def nn_descent(
    ^

  self.func_ir.loc))


Time in minutes:18.13346233765284
